# 🦠 Pandemic Commander
Welcome to the outbreak simulation!

You're in charge of managing a fictional disease outbreak. Your job is to tweak the parameters and try to complete three increasingly difficult public health challenges.

👇 Change the values in the code blocks, run the code, and check your results!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
%matplotlib inline

In [ ]:
def SIR_model(y, t, beta, gamma, vaccination_rate):
    S, I, R = y
    dSdt = -beta * S * I + vaccination_rate * R
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I - vaccination_rate * R
    return [dSdt, dIdt, dRdt]

In [ ]:
def simulate_sir(beta, gamma, vaccination_rate, label):
    S0, I0, R0 = 0.99, 0.01, 0.0
    y0 = [S0, I0, R0]
    global t, I  # make t and I accessible for checking duration later
    t = np.linspace(0, 160, 160)
    solution = odeint(SIR_model, y0, t, args=(beta, gamma, vaccination_rate))
    S, I, R = solution.T
    plt.plot(t, I, label=f'{label} (peak={max(I):.2f})')
    return max(I), R[-1], I[100]

## ✅ CHALLENGE 1: Strategic Flattening
Flatten the curve — but don’t stop the outbreak too early!
**Goal:**
- Peak infections must be **below 0.25**
- The outbreak must last **at least 60 days**

In [ ]:
# 🔧 Tweak these for CHALLENGE 1
beta = 0.3
gamma = 0.1
vaccination_rate = 0.00

plt.figure(figsize=(10, 5))
peak, final_R, I_day100 = simulate_sir(beta, gamma, vaccination_rate, 'Challenge 1')
plt.title('CHALLENGE 1: Strategic Flattening')
plt.xlabel('Time (days)')
plt.ylabel('Infected proportion')
plt.grid(True)
plt.legend()
plt.show()

duration = t[np.argmax(I < 0.001)] if np.any(I < 0.001) else 160
print(f'✅ Peak Infection = {peak:.2f}')
print(f'✅ Duration until infections fall below 0.001: {duration:.0f} days')
if peak < 0.25 and duration >= 60:
    print("🎉 SUCCESS: Strategic flattening achieved!")
else:
    print("🔁 Try again: Your curve was either too high or too short.")